In [1]:
# Disable jedi autocompleter
%config Completer.use_jedi = False


In [2]:
import numpy as np
import torch

In [3]:
from DDPGfD import DDPGfD
# from main_DDPGfD import state_dim_setup

# todo: get rid of jedi lol

In [4]:
torch.cuda.is_available()

True

In [5]:
def state_dim_setup(state_dim_option):
    """
    Returns an array of indices that can be used on a full observation to only grab relevant state dimensions
    Input: The argument of which state_range option to use.
    Output: Numpy array of indices
    """

    # Setup state dimensional parts here
    '''
    Local obs, all in local coordinates (from the center of the palm)
    (18,) Finger Pos                                        0-17: (0: x, 1: y, 2: z) "f1_prox", (3-5) "f2_prox", (6-8) "f3_prox", (9-11) "f1_dist", (12-14) "f2_dist", (15-17) "f3_dist"
    (3,) Wrist Pos                                          18-20 (18: x, 19: y, 20: z)
    (3,) Obj Pos                                            21-23 (21: x, 22: y, 23: z)
    (9,) Joint States                                       24-32
    (3,) Obj Size                                           33-35
    (12,) Finger Object Distance                            36-47
    
    36) "f1_prox"
    37) "f1_prox_1"
    38) "f2_prox"
    39) "f2_prox_1"
    40) "f3_prox"
    41) "f3_prox_1"
    42) "f1_dist"
    43) "f1_dist_1"
    44) "f2_dist"
    45) "f2_dist_1"
    46) "f3_dist"
    47) "f3_dist_1"
    
    Note: NONE vs "_1" meaning: On each finger there are two red dots. The "_1" is the ones closer to the center
    
    
    (2,) X and Z angle                                      48-49
    (17,) Rangefinder data                                  50-66
    (3,) Gravity vector in local coordinates                67-69
    (3,) Object location based on rangefinder data          70-72
    (1,) Ratio of the area of the side of the shape to the open portion of the side of the hand    73
    (1,) Ratio of the area of the top of the shape to the open portion of the top of the hand    74
    (6, ) Finger dot product  75) "f1_prox", 76) "f2_prox", 77) "f3_prox", 78) "f1_dist", 79) "f2_dist", 80) "f3_dist"  75-80
    (1, ) Dot product (wrist) 81
    '''

    finger_pos_idx = np.arange(0, 18)
    f1_prox_pos_idx = np.array([3, 4, 5])
    f2_prox_pos_idx = np.array([6, 7, 8])
    f1_dist_pos_idx = np.array([12, 13, 14])
    f2_dist_pos_idx = np.array([15, 16, 17])
    last_6_joint_states_idx = np.arange(27, 33)
    
    wrist_pos_idx = np.arange(18, 21)
    obj_pos_idx = np.arange(21, 24)
    joint_states_idx = np.arange(24, 33)
    obj_size_idx = np.arange(33, 36)
    finger_obj_dist_idx = np.arange(36, 48)
    
    finger_obj_dist_f1_prox_1 = np.array([37])
    finger_obj_dist_f2_prox_1 = np.array([39])
    finger_obj_dist_f1_dist_1 = np.array([43])
    finger_obj_dist_f2_dist_1 = np.array([45])
    
    
    x_z_angle_idx = np.arange(48, 50)
    rangefinder_data_idx = np.arange(50, 67)
    gravity_vector_in_local_coords = np.arange(67, 70)
    object_location_rangefinder = np.arange(70, 73)
    ratio_sideshape_sidehand = np.array([73])
    ratio_topshape_tophand = np.array([74])
    f1_prox_idx = np.array([75])
    f2_prox_idx = np.array([76])
    f3_prox_idx = np.array([77])
    f1_dist_idx = np.array([78])
    f2_dist_idx = np.array([79])
    f3_dist_idx = np.array([80])
    dot_prod_wrist = np.array([81])

    # create mappings for state dimension mapping
    state_dim_idx_arr_dict = {
        'all': np.arange(82),
        'nigel_rangefinder': np.concatenate((obj_pos_idx, rangefinder_data_idx, obj_size_idx), axis=0),
        'nigel_norangefinder': np.concatenate((obj_pos_idx, finger_obj_dist_idx, obj_size_idx), axis=0),
        'all_real': np.concatenate((f1_prox_pos_idx, f2_prox_pos_idx, f1_dist_pos_idx, f2_dist_pos_idx, obj_pos_idx, last_6_joint_states_idx, obj_size_idx, finger_obj_dist_idx)),
        #  wrist 3 + finger pos 12 + obj size 3 + last joint states 6 + obj pos 3 + finger obj dist 4
        'adam_sim2real': np.concatenate((f1_dist_pos_idx, f1_prox_pos_idx, f2_dist_pos_idx, f2_prox_pos_idx, wrist_pos_idx, obj_pos_idx, last_6_joint_states_idx, obj_size_idx, finger_obj_dist_f1_dist_1, finger_obj_dist_f1_prox_1, finger_obj_dist_f2_dist_1, finger_obj_dist_f2_prox_1))  # this one is based on sim2real
    }
    
    
    assert state_dim_option in state_dim_idx_arr_dict.keys()
    
#     # x_z useless
#     1. dont use finger 3, both in control and for position
#     2. for joint states - you only want the last 6

    res_state_idx_arr = state_dim_idx_arr_dict[state_dim_option]

    return res_state_idx_arr

In [6]:
state_idx_arr = state_dim_setup('adam_sim2real')
modified_state_dim = len(state_idx_arr)

# Set dimensions for state and action spaces - policy initialization
state_dim = 82  # State dimension dependent on the length of the state space
action_dim = 3 #env.action_space.shape[0]
max_action = 3.0
# max_action_trained = env.action_space.high  # a vector of max actions
# n = 5   # n step look ahead for the policy
# max_q_value = 50  # Should match the maximum reward value
# velocities = {"constant_velocity": 2, "min_velocity": 0, "max_velocity": 3, "finger_lift_velocity": 1, "wrist_lift_velocity": 1}


In [7]:
kwargs = {
        "state_dim": modified_state_dim,
        "action_dim": action_dim,
        "max_action": max_action,
#         "n": n,
#         "discount": args.discount,
#         "tau": args.tau,
#         "batch_size": args.batch_size,
#         "expert_sampling_proportion": args.expert_prob
    }

In [8]:
policy = DDPGfD(**kwargs)

================================ INITTING DDPGfD with state dim of:  31 ===============================


In [9]:
trained_policy_path = '/home/mechagodzilla/sim-to-real-kinova/src/sim-to-real-kinova-master/openai_gym_kinova/src/policies/state_dim_full_train_v01/_07_22_21_0544/policy/train_DDPGfD_kinovaGrip'
trained_policy_path = 'policies/state_dim_full_train_v01/_07_22_21_0544/policy/train_DDPGfD_kinovaGrip'
policy.load(trained_policy_path)

In [10]:
fake_state = np.zeros((31)) + np.random.normal(size=(31))
print('fake state shape:', fake_state.shape)
policy.select_action(fake_state)

fake state shape: (31,)


array([0.6577568 , 0.69768643, 0.32741684], dtype=float32)

In [18]:
def state_dim_setup(state_dim_option):
    """
    Returns an array of indices that can be used on a full observation to only grab relevant state dimensions
    Input: The argument of which state_range option to use.
    Output: Numpy array of indices
    """

    # Setup state dimensional parts here
    '''
    Local obs, all in local coordinates (from the center of the palm)
    (18,) Finger Pos                                        0-17: (0: x, 1: y, 2: z) "f1_prox", (3-5) "f2_prox", (6-8) "f3_prox", (9-11) "f1_dist", (12-14) "f2_dist", (15-17) "f3_dist"
    (3,) Wrist Pos                                          18-20 (18: x, 19: y, 20: z)
    (3,) Obj Pos                                            21-23 (21: x, 22: y, 23: z)
    (9,) Joint States                                       24-32
    (3,) Obj Size                                           33-35
    (12,) Finger Object Distance                            36-47
    
    36) "f1_prox"
    37) "f1_prox_1"
    38) "f2_prox"
    39) "f2_prox_1"
    40) "f3_prox"
    41) "f3_prox_1"
    42) "f1_dist"
    43) "f1_dist_1"
    44) "f2_dist"
    45) "f2_dist_1"
    46) "f3_dist"
    47) "f3_dist_1"
    
    Note: NONE vs "_1" meaning: On each finger there are two red dots. The "_1" is the ones closer to the center
    
    
    (2,) X and Z angle                                      48-49
    (17,) Rangefinder data                                  50-66
    (3,) Gravity vector in local coordinates                67-69
    (3,) Object location based on rangefinder data          70-72
    (1,) Ratio of the area of the side of the shape to the open portion of the side of the hand    73
    (1,) Ratio of the area of the top of the shape to the open portion of the top of the hand    74
    (6, ) Finger dot product  75) "f1_prox", 76) "f2_prox", 77) "f3_prox", 78) "f1_dist", 79) "f2_dist", 80) "f3_dist"  75-80
    (1, ) Dot product (wrist) 81
    '''

    finger_pos_idx = np.arange(0, 18)
    f1_prox_pos_idx = np.array([3, 4, 5])
    f2_prox_pos_idx = np.array([6, 7, 8])
    f1_dist_pos_idx = np.array([12, 13, 14])
    f2_dist_pos_idx = np.array([15, 16, 17])
    last_6_joint_states_idx = np.arange(27, 33)
    
    wrist_pos_idx = np.arange(18, 21)
    obj_pos_idx = np.arange(21, 24)
    joint_states_idx = np.arange(24, 33)
    obj_size_idx = np.arange(33, 36)
    finger_obj_dist_idx = np.arange(36, 48)
    
    finger_obj_dist_f1_prox_1 = np.array([37])
    finger_obj_dist_f2_prox_1 = np.array([39])
    finger_obj_dist_f1_dist_1 = np.array([43])
    finger_obj_dist_f2_dist_1 = np.array([45])
    
    
    x_z_angle_idx = np.arange(48, 50)
    rangefinder_data_idx = np.arange(50, 67)
    gravity_vector_in_local_coords = np.arange(67, 70)
    object_location_rangefinder = np.arange(70, 73)
    ratio_sideshape_sidehand = np.array([73])
    ratio_topshape_tophand = np.array([74])
    f1_prox_idx = np.array([75])
    f2_prox_idx = np.array([76])
    f3_prox_idx = np.array([77])
    f1_dist_idx = np.array([78])
    f2_dist_idx = np.array([79])
    f3_dist_idx = np.array([80])
    dot_prod_wrist = np.array([81])

    # create mappings for state dimension mapping
    state_dim_idx_arr_dict = {
        'all': np.arange(82),
        'nigel_rangefinder': np.concatenate((obj_pos_idx, rangefinder_data_idx, obj_size_idx), axis=0),
        'nigel_norangefinder': np.concatenate((obj_pos_idx, finger_obj_dist_idx, obj_size_idx), axis=0),
        'all_real': np.concatenate((f1_prox_pos_idx, f2_prox_pos_idx, f1_dist_pos_idx, f2_dist_pos_idx, obj_pos_idx, last_6_joint_states_idx, obj_size_idx, finger_obj_dist_idx)),
        #  wrist 3 + finger pos 12 + obj size 3 + last joint states 6 + obj pos 3 + finger obj dist 4
        'adam_sim2real': np.concatenate((f1_dist_pos_idx, f1_prox_pos_idx, f2_dist_pos_idx, f2_prox_pos_idx, wrist_pos_idx, obj_pos_idx, last_6_joint_states_idx, obj_size_idx, finger_obj_dist_f1_dist_1, finger_obj_dist_f1_prox_1, finger_obj_dist_f2_dist_1, finger_obj_dist_f2_prox_1))  # this one is based on sim2real
    }
    
    assert state_dim_option in state_dim_idx_arr_dict.keys()

    res_state_idx_arr = state_dim_idx_arr_dict[state_dim_option]

    return res_state_idx_arr
    
    
def lerp(action_arr, old_min=0, old_max=3, new_min=0, new_max=6800):
    # first: scale to proper min max
    np_arr = np.array(action_arr)
    scale_factor = (new_max - new_min) / (old_max - old_min)
    scaled_arr = (np_arr - old_min) * scale_factor + new_min
    return scaled_arr

# assume state_dim_setup has been imported, probably from a utilities file.

class Agent:
    """
    base action class
    """

    def __init__(self):
        return

    def act(self, obs):
        return

class RLAgent(Agent):
    """
    action class based on trained RL agent
    """

    def __init__(self, trained_policy_path):
        super(Agent, self).__init__()
        
        state_idx_arr = state_dim_setup('adam_sim2real')
        modified_state_dim = len(state_idx_arr)

        # Set dimensions for state and action spaces - policy initialization
        state_dim = 82  # State dimension dependent on the length of the state space
        action_dim = 3  # env.action_space.shape[0]
        self.max_action = 3.0  # this is hardcoded from our simulation environment
        
        self.max_action_real_world = 6800  # this is hardcoded from real life
        
        kwargs = {
        "state_dim": modified_state_dim,
        "action_dim": action_dim,
        "max_action": max_action
        }
        
        self.policy = DDPGfD(**kwargs)
        
        self.policy.load(trained_policy_path)

    def act(self, obs):
        """
        lmao pick some random shit
        """
        print('observation shape:', obs.shape)
        rl_action = self.policy.select_action(obs)
        scaled_action = lerp(rl_action, old_min=-self.max_action, old_max=self.max_action,
                             new_min=-self.max_action_real_world,
                             new_max=self.max_action_real_world)
        return scaled_action

In [12]:
agent = RLAgent(trained_policy_path=trained_policy_path)

================================ INITTING DDPGfD with state dim of:  31 ===============================


In [14]:
agent.act(fake_state)

observation shape: (31,)


array([1490.916 , 1581.4229,  742.145 ], dtype=float32)

In [20]:
import os

In [50]:
import yaml

config_path = 'experiment_configs/'
filename = 'constant_speed_test.yaml'
config_filepath = os.path.join(config_path, filename)

stream = open(config_filepath, 'r')

config_dict = yaml.load(stream, Loader=yaml.FullLoader)
config_dict

{'experiment_name': 'constant_speed_test',
 'params': {'controller': {'type': 'constant',
   'speed': 6800,
   'options': [0, 1700, 3400],
   'distribution': 'uniform',
   'policy_filepath': 'policies/state_dim_full_train_v01/_07_22_21_0544/policy/train_DDPGfD_kinovaGrip'},
  'logger': {'use_logger': True,
   'use_video': True,
   'log_dir': 'constant_agent_logs'},
  'noise': {'noise_range': 'fixed',
   'noise_distribution': 'uniform',
   'x_noise': [0.03],
   'y_noise': [-0.02],
   'z_noise': [0.0],
   'roll_noise': [0.0],
   'pitch_noise': [0.0],
   'yaw_noise': [-0.261799]},
  'max_timesteps': 100}}

In [47]:
experiment_name = config_dict['experiment_name']
experiment_params = config_dict['params']

controller_params = experiment_params['controller']
logger_params = experiment_params['logger']
noise_params = experiment_params['noise']
max_timesteps = experiment_params['max_timesteps']

In [48]:
controller_type = controller_params['type']
if controller_type == 'constant':
    agent = ConstantAgent(speed=controller_params['speed'])
elif controller_type == 'discrete_random':
    agent = RandomAgent(options=controller_params)
elif controller_type == 'rl':
    agent = RLAgent(trained_policy_path=controller_params['policy_filepath'])

logger = None
if logger_params['use_logger']:
    log_dir = os.path.join(rel_dirname, logger_params['log_dir'])
    logger = Logger(log_dir=log_dir, use_video=logger_params['use_video'])
    
noiser = Noiser(noise_params['x_noise'],noise_params['x_noise'],noise_params['x_noise'],noise_params['x_noise'],noise_params['x_noise'],noise_params['x_noise'],noise_range=noise_params['noise_range'],noise_distribution=noise_params['noise_distribution'])


NameError: name 'ConstantAgent' is not defined

In [49]:
class Noiser:
    """
    Adds some fucking noise
    """
    def __init__(self, x_noise: List, y_noise: List, z_noise: List, roll_noise: List, pitch_noise: List, yaw_noise: List, noise_range='fixed', noise_distribution='uniform'):
        self.x_noise = x_noise
        self.y_noise = y_noise
        self.z_noise = z_noise
        self.roll_noise = roll_noise
        self.pitch_noise = pitch_noise
        self.yaw_noise = yaw_noise
        
        self.noise_range = noise_range
        assert self.noise_range in ['random', 'fixed'], 'your noise range is wrong. see: ' + self.noise_range
        
        if self.noise_range == 'random':
            self.noise_distribution = noise_distribution
            assert self.noise_distribution in ['uniform', 'normal'], 'your noise distribution is wrong. see: ' + self.noise_distribution
            # and then assume the lists are length 2
            for noise_arr in [x_noise, y_noise, z_noise, roll_noise, pitch_noise, yaw_noise]:
                assert len(noise_arr) == 2, 'one of your noise arrays is not length 2. should be a min and max'
        elif self.noise_range == 'fixed':
            # go through every permutation
            self.noise_permutation_counter = 0
            self.reset_counter = 0
            
            # this is doggy doo list comprehension
            self.noise_permutation_arr = [[x, y, z, roll, pitch, yaw] for x in x_noise for y in y_noise for z in z_noise for roll in roll_noise for pitch in pitch_noise for yaw in yaw_noise]
            
    def sample_noise(self):
        if self.noise_range == 'random':
            
            if self.noise_distribution == 'uniform':
                x = np.random.uniform(low=self.x_noise[0], high=self.x_noise[1])
                y = np.random.uniform(low=self.y_noise[0], high=self.y_noise[1])
                z = np.random.uniform(low=self.z_noise[0], high=self.z_noise[1])
                roll = np.random.uniform(low=self.roll_noise[0], high=self.roll_noise[1])
                pitch = np.random.uniform(low=self.pitch_noise[0], high=self.pitch_noise[1])
                yaw = np.random.uniform(low=self.yaw_noise[0], high=self.yaw_noise[1])
                
            elif self.noise_distribution == 'normal':
                # oop write this later
                x = np.random.choice(self.x_noise)
                y = np.random.choice(self.y_noise)
                z = np.random.choice(self.z_noise)
                roll = np.random.choice(self.roll_noise)
                pitch = np.random.choice(self.pitch_noise)
                yaw = np.random.choice(self.yaw_noise)
            
            return [x, y, z, roll, pitch, yaw]
            
        elif self.noise_range == 'fixed':
            if self.noise_permutation_counter == len(self.noise_permutation_arr):
                self.noise_permutation_counter = 0
                self.reset_counter += 1
                
            # grab the permutation
            noise_arr = self.noise_permutation_arr[self.noise_permutation_counter]
            
            # increment position in permutation arr
            self.noise_permutation_counter += 1
            
            return noise_arr, self.noise_permutation_counter, self.reset_counter
            

NameError: name 'List' is not defined

In [ ]:
:L

In [9]:
import gym

env = gym.make('gym_kinova_gripper:kinovagripper-v0')



In [10]:
for i in range(100):
    env.render()
    
env.close() # broken

Creating window glfw


In [6]:
env.close()

In [9]:
env.step([0, 0, 0])

IndexError: list index out of range

In [5]:
obs = env.reset(shape_keys=['CubeM'], hand_orientation='normal')  # TODO: make sure this is NO OBJECT
env.render()

NameError: name 'env' is not defined

In [13]:
# FIRST EXPERIMENT: ACTION SPACE

done = False

timestep_count = 0

# shape_keys=['CubeM', 'CubeS']
# hand_orientation: random, normal (0 deg), top (90 deg), rotated (67 deg)
obs = env.reset(shape_keys=['CubeM'], hand_orientation='normal')  # TODO: make sure this is NO OBJECT
env.render()

while not done:
    action = np.array([0, 3, 3, 3])
    obs, total_reward, done, info = env.step(action)
    
    env.render()
    
    timestep_count += 1

In [12]:
print('num of time steps:', timestep_count)



num of time steps: 45


In [19]:
idx_arr = state_dim_setup('adam_sim2real')

In [59]:
obs = env.reset(shape_keys=['CubeM'], hand_orientation='normal')  # TODO: make sure this is NO OBJECT
np.array(obs)[idx_arr]

array([ 7.81004252e-02,  3.42315167e-02,  3.13802628e-02,  4.98781332e-02,
        1.19782256e-02,  2.61084636e-02,  7.81004174e-02,  3.42315167e-02,
       -3.14348337e-02,  4.98780071e-02,  1.19782256e-02, -2.61636678e-02,
        0.00000000e+00,  1.02999206e-18, -1.38777878e-17,  3.51104606e-03,
        3.82424873e-02, -1.25666907e-02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        2.03710422e-02,  2.03710422e-02,  1.05462499e-01,  8.84488016e-02,
        5.56575851e-02,  8.94585565e-02,  5.94800385e-02])

In [86]:
obs, total_reward, done, info = env.step(action)
done, np.array(obs)[idx_arr]

(False,
 array([ 6.91348060e-03,  5.81679647e-02,  1.80836220e-02,  2.87007696e-02,
         2.78457039e-02,  2.21528468e-02,  2.60393180e-03,  5.58641107e-02,
        -1.73332389e-02,  2.70172910e-02,  2.77449727e-02, -2.18936268e-02,
         0.00000000e+00, -2.43945489e-18,  0.00000000e+00, -1.10534958e-02,
         2.53972089e-02, -1.13681256e-02,  9.33591052e-01,  1.30907301e+00,
         1.38715544e+00,  4.61123097e-01,  6.50463890e-01,  6.89746205e-01,
         2.03710422e-02,  2.03710422e-02,  1.05462499e-01,  4.55606385e-02,
         2.27877318e-02,  4.39041261e-02,  4.59486178e-02]))

In [88]:
import os
os.path.dirname(os.path.realpath(__file__))

NameError: name '__file__' is not defined